Less of a unit test, more an exploration of waffles.

In [45]:
filename = 'samples/waffle.tex'

In [50]:
import TexSoup
import re

In [47]:
with open(filename,'r') as f:
    tex_doc = f.read()

Sometimes there are weird errors like spaces in begins/ends -- fix these first:

In [49]:
search_weirdos_begin = r'\\begin(\s*){(\s*)[A-Za-z]*(\s*)}' # \begin {equation} or \end { document}, etc
search_weirdos_end = r'\\end(\s*){(\s*)[A-Za-z]*(\s*)}' # same for end

In [66]:
ind = 0
tex_doc_out = tex_doc
while ind < len(tex_doc):
    if re.search(search_weirdos_begin, tex_doc[ind:]):
        istart,iend = re.search(search_weirdos_begin, tex_doc[ind:]).span()
        text = tex_doc[ind:][istart:iend]
        text_replace = "".join(text.split())
        #print(text, text_replace)
        tex_doc_out = tex_doc.replace(text,text_replace)
        ind += iend
    else:
        ind += len(tex_doc[ind:])
        
tex_doc = tex_doc_out
ind = 0
while ind < len(tex_doc):
    if re.search(search_weirdos_end, tex_doc[ind:]):
        istart,iend = re.search(search_weirdos_end, tex_doc[ind:]).span()
        text = tex_doc[ind:][istart:iend]
        text_replace = "".join(text.split())
        tex_doc_out = tex_doc.replace(text,text_replace)
        ind += iend
    else:
        ind += len(tex_doc[ind:])
tex_doc = tex_doc_out

In [67]:
print(tex_doc)

\documentclass[a4paper]{article}

\begin{document}

\section{Waffles waffles waffles}

\subsection{Waffles are cooler than you}

Waffles have significantly more pockets than you.  

\[
P(W) > P(\rm{you})
\]

Ok, maybe you are at least as cool as waffles:

$$
C_{ool}(\rm{you}) \ge C_{ool}(W)
$$

Sometimes though, you might type things in silly ways $C_ {ool  } (\rm{anybody}) >   0$.

\begin{equation}
C_{   ool} \ne 0
\end{equation}

\end{document}



In [57]:
"".join('\\begin{\n document }'.split())

'\\begin{document}'

In [48]:
soup = TexSoup.TexSoup(r''+tex_doc, tolerance=0)

In [ ]:
print(tex_doc)

In [ ]:
print(soup)

In [32]:
len(soup.all)

5

In [34]:
for iss,s in enumerate(soup.all):
    print('----------- soup element =', iss, '---------------')
    print(type(s))
    print(type(s.expr))
    print(s)
    print('----------------')

----------- soup element = 0 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexCmd'>
\documentclass[a4paper]{article}
----------------
----------- soup element = 1 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>


----------------
----------- soup element = 2 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexText'>


----------------
----------- soup element = 3 ---------------
<class 'TexSoup.data.TexNode'>
<class 'TexSoup.data.TexNamedEnv'>
\begin{document}

\section{Waffles waffles waffles}

\subsection{Waffles are cooler than you}

Waffles have significantly more pockets than you.  

\[
P(W) > P(\rm{you})
\]

Ok, maybe you are at least as cool as waffles:

$$
C_{ool}(\rm{you}) \ge C_{ool}(W)
$$

Sometimes though, you might type things in silly ways $C_ {ool  } (\rm{anybody}) >   0$.

\begin{equation}
C_{   ool} \ne 0
\end{equation}

\end{document}
----------------
----------- soup element = 4 ---------------
<c

In [40]:
for iss,s in enumerate(soup.all):
    print('----------- soup element =', iss, '---------------')
    if type(s.expr) == TexSoup.data.TexNamedEnv: # parts of the text
        if 'begin' in s.expr.begin and 'document' in s.expr.begin:
            for isss,ss in enumerate(s.all):
                print('----------- sub-soup element =', isss, '---------------')
                #print(type(ss))
                print(type(ss.expr))
                print(ss)
                print('-----------------')
        else:
            print(' ******** not in document *********** ')

----------- soup element = 0 ---------------
----------- soup element = 1 ---------------
----------- soup element = 2 ---------------
----------- soup element = 3 ---------------
----------- sub-soup element = 0 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 1 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 2 ---------------
<class 'TexSoup.data.TexCmd'>
\section{Waffles waffles waffles}
-----------------
----------- sub-soup element = 3 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 4 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 5 ---------------
<class 'TexSoup.data.TexCmd'>
\subsection{Waffles are cooler than you}
-----------------
----------- sub-soup element = 6 ---------------
<class 'TexSoup.data.TexText'>


-----------------
----------- sub-soup element = 7 ---------------
<c